# Query DVC Experiments and create Latex Table

In [ ]:
# ruff: noqa

In [ ]:
import dvc.api
import pandas as pd

pd.set_option("display.max_columns", 500)

In [ ]:
exps = dvc.api.exp_show(revs="e7d052c")

In [ ]:
# tags = dvc.api.scm.all_tags()
# exps = dvc.api.exp_show(revs=tags)
df = pd.DataFrame(exps)
df = df.dropna(subset=["Experiment"]).reset_index(drop=True)
df.head(10)

In [ ]:
# Create a function to format metrics as mean (±std) with highlighting
def format_table_with_metrics_and_highlighting(df, target="energy"):
    # Define display name mapping
    display_names = {
        "pinball_loss": "Pinball Loss",
        "interval_score_50": r"$\text{IS}_{0.5}$",
        "interval_score_95": r"$\text{IS}_{0.05}$",
        "coverage_50": r"$\text{Cvg}_{0.5}$",
        "coverage_95": r"$\text{Cvg}_{0.05}$",
    }

    # Create new formatted DataFrame
    formatted_table = pd.DataFrame()
    formatted_table["Model"] = df[f"train.{target}.selected"]

    # Format metrics with highlighting
    for metric in ["pinball_loss", "interval_score_50", "interval_score_95"]:
        means = df[f"{metric}.mean"]
        stds = df[f"{metric}.std"]
        min_idx = means.idxmin()

        values = []
        for idx in means.index:
            if idx == min_idx:
                values.append(f"\\textbf{{{means[idx]:.2f} (± {stds[idx]:.2f})}}")
            else:
                values.append(f"{means[idx]:.2f} (± {stds[idx]:.2f})")
        formatted_table[display_names[metric]] = values

    # Handle coverage metrics
    for metric, target in [("coverage_50", 0.5), ("coverage_95", 0.95)]:
        means = df[f"{metric}.mean"]
        stds = df[f"{metric}.std"]
        # Find index closest to target
        closest_idx = (means - target).abs().idxmin()

        values = []
        for idx in means.index:
            if idx == closest_idx:
                values.append(f"\\textbf{{{means[idx]:.2f} (± {stds[idx]:.2f})}}")
            else:
                values.append(f"{means[idx]:.2f} (± {stds[idx]:.2f})")
        formatted_table[display_names[metric]] = values

    return formatted_table

In [ ]:
def format_dvc_experiments(
    exps: dict,
    target="energy",
    allow_duplicated_models=False,
    caption: str | None = None,
    label: str | None = None,
) -> tuple[pd.DataFrame, pd.DataFrame, str]:
    df = pd.DataFrame(exps)
    df = df.dropna(subset=["Experiment"]).reset_index(drop=True)
    metrics = [col for col in df.columns if f"{target}_metrics.json" in col]
    params_col = [f"train.{target}.selected"]

    df = pd.DataFrame(exps, columns=metrics + params_col)
    df = df.dropna()
    if not allow_duplicated_models:
        df = df.drop_duplicates(subset=params_col)

    # strip the prefix in the col name until :
    df.columns = [col.split(":")[1] if ":" in col else col for col in df.columns]
    df = df.drop(columns=["avg_fit_time", "avg_pred_time"])
    df = df.sort_values("pinball_loss.mean", ascending=False, ignore_index=True)

    # adjust names of models
    df[f"train.{target}.selected"] = df[f"train.{target}.selected"].replace(
        {
            "lgbm": "LightGBM",
            "xgb-custom": "XGBoost",
            "catboost": "CatBoost",
            "quantreg": "Quantile Regression",
            "benchmark": "Benchmark",
        }
    )
    print(df)
    formatted_table = format_table_with_metrics_and_highlighting(df, target=target)
    # formatted_table = formatted_table.drop(
    #     columns=[r"50\% Interval Score", r"95\% Interval Score"]
    # )

    latex_code = formatted_table.to_latex(
        index=False,
        escape=False,
        caption=caption,
        label=label,
        position="htp",
        column_format="lrrrrr",
    )
    return df, formatted_table, latex_code

In [ ]:
df_bikes, _, bikes_table = format_dvc_experiments(
    exps,
    target="bikes",
    caption="Results of Timeseries Cross-Validation on the Daily Bike Count Dataset. Best values are highlighted in bold.",
    label="tab:bikes_results",
    allow_duplicated_models=True,
)
df_bikes

In [ ]:
_, _, energy_table = format_dvc_experiments(
    exps,
    target="energy",
    caption="Results of Timeseries Cross-Validation on the Hourly Electricity Demand Dataset. Best values are highlighted in bold.",
    label="tab:energy_results",
    allow_duplicated_models=True,
)

In [ ]:
print(bikes_table)

In [ ]:
print(energy_table)